In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import numpy as np

In [36]:
df = pd.read_csv('osm_places_amenity.csv')

threshold = int(0.03* df.shape[0])
df = df.dropna(axis=1, thresh=threshold)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_3128\3000964665.py:1: DtypeWarning: Columns (47,70) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('osm_places_amenity.csv')


In [37]:
df.columns

Index(['id', 'lat', 'lon', 'addr:city', 'addr:housenumber', 'addr:postcode',
       'addr:state', 'addr:street', 'amenity', 'branch', 'brand',
       'brand:wikidata', 'cuisine', 'drive_through', 'name', 'official_name',
       'opening_hours', 'phone', 'takeaway', 'website', 'outdoor_seating',
       'cocktails', 'drink:beer', 'drink:liquor', 'drink:wine', 'wheelchair',
       'check_date', 'payment:cash', 'payment:credit_cards', 'email',
       'alt_name', 'ref', 'internet_access', 'level', 'diet:vegan',
       'diet:vegetarian', 'toilets', 'drink:coffee', 'drink:tea', 'smoking',
       'bar', 'contact:instagram', 'delivery', 'drink:espresso', 'reservation',
       'indoor_seating'],
      dtype='object')

In [38]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [39]:
df = df[['id', 'lat', 'lon', 'addr:city', 'addr:housenumber', 'addr:postcode',
       'addr:state', 'addr:street', 'amenity', 'cuisine', 'drive_through', 'name',
       'opening_hours', 'phone', 'website', 'outdoor_seating',
       'cocktails', 'drink:beer', 'drink:liquor', 'drink:wine', 'wheelchair',
       'email', 'diet:vegan',
       'diet:vegetarian', 'drink:coffee', 'drink:tea', 
       'contact:instagram']]

In [40]:
df.head()

,id,lat,lon,addr:city,addr:housenumber,addr:postcode,addr:state,addr:street,amenity,cuisine,drive_through,name,opening_hours,phone,website,outdoor_seating,cocktails,drink:beer,drink:liquor,drink:wine,wheelchair,email,diet:vegan,diet:vegetarian,drink:coffee,drink:tea,contact:instagram
0,349323821,40.762515,-73.976690,New York,43,10019,NY,West 55th Street,fast_food,burger,no,Five Guys,Mo-Su 11:00-22:00,+1 212-459-9600,https://fiveguys.olo.express/menu/nyc-55th-st/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,357618253,40.740822,-73.784295,Fresh Meadows,61-09,11365,NY,190th Street,restaurant,burger,NaN,Hooters,NaN,(718) 454-2800,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,357620442,40.741558,-73.978463,NaN,NaN,NaN,NaN,NaN,restaurant,korean,NaN,Sam Sunny,Mo-Th 12:00-22:00; Fr-Sa 12:00-23:00; Su 12:00...,+1-212-837-1044,https://www.samsunnynyc.com,NaN,yes,draught,yes,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,357621192,40.846563,-73.938127,New York,4140,10033,NY,Broadway,theatre,NaN,NaN,United Palace,NaN,NaN,https://www.unitedpalace.org/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,368042980,40.721480,-73.997103,NaN,97,10012,NaN,Kenmare Street,arts_centre,NaN,NaN,Storefront for Art and Architecture,NaN,+1-212-431-5795,https://storefrontnews.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# Drop missing name values
df.drop(df[df['name'].isna()].index, inplace=True)

In [42]:
# Change vegan/vegetarian only/limited values
df.replace({'diet:vegan':'limited','diet:vegetarian': 'limited'}, 'yes', inplace=True)
df.replace({'diet:vegan':'only','diet:vegetarian': 'only'}, 'yes', inplace=True)

In [43]:
df['diet:vegan'].unique()

array([nan, 'yes', 'no'], dtype=object)

In [44]:
# Replace NaN values with "no" for vegan, vegetarian, wheelchair, outdoor seating, cocktails, beer, liquor, wine, coffee, tea
df.fillna({"diet:vegan":"unknown", "diet:vegetarian":"unknown", 'wheelchair':"unknown", 'outdoor_seating':"unknown", 'cocktails':"unknown", 'drink:beer':"unknown", 'drink:liquor':"unknown", 'drink:wine':"unknown", 'drink:coffee':"unknown", 'drink:tea':"unknown"}, inplace=True)


In [45]:
# Drop cols with NJ in State
df.drop(df[df['addr:state'] == 'NJ'].index, inplace=True)

In [46]:
df['wheelchair'].unique()

array(['unknown', 'yes', 'limited', 'no', 'designated', 'assisted',
       'dedicated'], dtype=object)

In [47]:
# Change designated and dedicated values to yes for wheelchair
df.replace({'wheelchair':'designated'}, 'yes', inplace=True)
df.replace({'wheelchair':'dedicated'}, 'yes', inplace=True)

In [48]:
# Replace non "no" values with "yes" for outdoor seating, cocktails, beer, liquor, wine, coffee, tea
cols = ["diet:vegan", "diet:vegetarian", 'wheelchair', 'outdoor_seating', 'cocktails', 'drink:beer', 'drink:liquor', 'drink:wine', 'drink:coffee', 'drink:tea']

for col in cols:
    df.loc[~(df[col].isin(['no', 'unknown'])), col] = "yes"

In [49]:
df.head()

,id,lat,lon,addr:city,addr:housenumber,addr:postcode,addr:state,addr:street,amenity,cuisine,drive_through,name,opening_hours,phone,website,outdoor_seating,cocktails,drink:beer,drink:liquor,drink:wine,wheelchair,email,diet:vegan,diet:vegetarian,drink:coffee,drink:tea,contact:instagram
0,349323821,40.762515,-73.976690,New York,43,10019,NY,West 55th Street,fast_food,burger,no,Five Guys,Mo-Su 11:00-22:00,+1 212-459-9600,https://fiveguys.olo.express/menu/nyc-55th-st/...,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown,unknown,unknown,NaN
1,357618253,40.740822,-73.784295,Fresh Meadows,61-09,11365,NY,190th Street,restaurant,burger,NaN,Hooters,NaN,(718) 454-2800,NaN,yes,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown,unknown,unknown,NaN
2,357620442,40.741558,-73.978463,NaN,NaN,NaN,NaN,NaN,restaurant,korean,NaN,Sam Sunny,Mo-Th 12:00-22:00; Fr-Sa 12:00-23:00; Su 12:00...,+1-212-837-1044,https://www.samsunnynyc.com,unknown,yes,yes,yes,yes,unknown,NaN,unknown,unknown,unknown,unknown,NaN
3,357621192,40.846563,-73.938127,New York,4140,10033,NY,Broadway,theatre,NaN,NaN,United Palace,NaN,NaN,https://www.unitedpalace.org/,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown,unknown,unknown,NaN
4,368042980,40.721480,-73.997103,NaN,97,10012,NaN,Kenmare Street,arts_centre,NaN,NaN,Storefront for Art and Architecture,NaN,+1-212-431-5795,https://storefrontnews.org,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown,unknown,unknown,NaN


In [50]:
df['addr:postcode'] = df['addr:postcode'].str[:5]

In [51]:
df.to_csv("osm_places_amenity_cleaned.csv", index=False)